In [31]:
import joblib
le = joblib.load("denial_reason_encoder_grouped.pkl")
print(le.classes_)


['Duplicate Claim Submission' 'Eligibility Verification Failure'
 'Inadequate Clinical Documentation' 'Incomplete Registration Data'
 'Incorrect Procedure Codes' 'Other']


In [29]:
import pandas as pd
import numpy as np
import joblib

# Load new data
df = pd.read_csv("new_claims.csv")

# Load encoders and models
label_encoders = joblib.load("label_encoders_status.pkl")
status_encoder = joblib.load("claim_status_encoder.pkl")
model = joblib.load("claim_status_model.pkl")

# Drop unused columns
df_encoded = df.copy()
drop_cols = ['claim_id', 'patient_id', 'provider_id', 'service_date', 'claim_submission_date']
df_encoded.drop(columns=drop_cols, inplace=True)

# Encode object columns using label encoders
for col in df_encoded.select_dtypes(include='object').columns:
    le = label_encoders.get(col)
    if le:
        df_encoded[col] = df_encoded[col].map(lambda x: x if x in le.classes_ else le.classes_[0])
        le.classes_ = np.unique(np.append(le.classes_, df_encoded[col].unique()))
        df_encoded[col] = le.transform(df_encoded[col])

# Predict claim status
df['predicted_status'] = status_encoder.inverse_transform(model.predict(df_encoded))

# Handle denial reason prediction for 'Denied' claims
denied_indices = df[df['predicted_status'] == 'Denied'].index

if not denied_indices.empty:
    # Load denial reason model and encoder
    denial_model = joblib.load("denial_reason_model_grouped.pkl")
    denial_encoder = joblib.load("denial_reason_encoder_grouped.pkl")

    # Add 'is_denied' column with value 1
    df_encoded.loc[denied_indices, 'is_denied'] = 1

    # Ensure all required columns for denial model are present
    expected_cols = denial_model.get_booster().feature_names
    for col in expected_cols:
        if col not in df_encoded.columns:
            df_encoded[col] = 0  # fill missing columns if needed

    # Reorder columns to match model expectation
    df_encoded = df_encoded[expected_cols]

    # Predict and decode denial reasons
    denial_preds = denial_model.predict(df_encoded.loc[denied_indices])
    denial_reasons = denial_encoder.inverse_transform(denial_preds.astype(int))
    df.loc[denied_indices, 'predicted_denial_reason'] = denial_reasons
else:
    df['predicted_denial_reason'] = None

# Save results
df.to_csv("predicted_claims.csv", index=False)
print("✅ Prediction results saved to 'predicted_claims.csv'")


✅ Prediction results saved to 'predicted_claims.csv'


In [38]:
denied = df3[df3['predicted_status']=='Denied']
denied

,claim_id,patient_id,patient_age,patient_gender,specialty,provider_id,procedure_code,diagnosis_code,service_date,claim_submission_date,claim_amount,insurance_provider,predicted_status,predicted_denial_reason
5,6456291,320538,34,Male,Neurology,85791,99211,R10.9,2024-12-27,2025-02-16,23898.48,Cigna,Denied,Other
6,1508857,47360,22,Female,Infectious Diseases,34518,90471,F32.9,2024-03-29,2025-06-03,10422.44,Blue Cross Blue Shield,Denied,Incomplete Registration Data
7,1810923,693295,85,Other,Diabetes and Endocrinology,20874,99212,E11.9,2024-10-24,2025-03-06,19972.08,Kaiser Permanente,Denied,Other
11,460524,558587,60,Female,Gastroenterology,76550,99211,I10,2024-07-28,2024-12-19,36183.35,Cigna,Denied,Other
18,5513929,841564,29,Male,OB-GYN,46031,99213,K21.9,2023-09-17,2024-09-06,18935.20,Blue Cross Blue Shield,Denied,Other
23,2150694,889525,55,Female,Rheumatology,36767,99203,K21.9,2024-03-03,2025-02-21,48748.99,Blue Cross Blue Shield,Denied,Inadequate Clinical Documentation
26,4585522,627934,75,Female,OB-GYN,47579,99213,M17.9,2024-05-01,2025-04-26,26127.38,Kaiser Permanente,Denied,Eligibility Verification Failure
28,8815675,415721,65,Male,Dermatology,16960,99212,I10,2024-09-27,2024-12-09,13660.39,Blue Cross Blue Shield,Denied,Other
29,4226249,804071,49,Male,Infectious Diseases,93658,99204,N18.9,2024-05-26,2024-08-11,2246.93,Blue Cross Blue Shield,Denied,Other
31,2511862,297523,47,Male,Internal Medicine,34293,99212,I10,2024-11-10,2025-04-02,17123.10,Blue Cross Blue Shield,Denied,Other
